In [1]:
import hopsworks

In [2]:
project = hopsworks.login(
    host="2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai",
    project="electricity",
    api_key_value="xIGaDiwBZpXxLYRQ.SgheZwqHTZteASnO1PtPMC4HMgco2OVD6VK6xZN3sKCy96FQT27rF0JbBuFivTTA",
)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287
Connected. Call `.close()` to terminate connection gracefully.


In [15]:
se_power_gen_fg = fs.get_feature_group("se_power_generation")
se_cross_region_flows_fg = fs.get_feature_group("se_cross_region_flows")
se_holidays_fg = fs.get_feature_group("se_holidays")
se_weather_fg = fs.get_feature_group("se_weather_fg")

se_prices_fg = fs.get_feature_group("se_da_prices")

In [37]:
query = se_prices_fg.select_except(['zone'])\
    .join(se_weather_fg.select_except(['zone', 'time', 'precipitaton_type_se3']))\
    .join(se_holidays_fg.select_except(['zone', 'day']))\
    .join(se_cross_region_flows_fg.select_except(['zone', 'time', 'dk1no1se3', 'se3dk1no1']))\
    .join(se_power_gen_fg.select_except(['zone', 'mtu', 'marine']))

In [40]:
transformations = {
    'mean_temp_per_day_se3': fs.get_transformation_function("min_max_scaler"),
    'mean_wind_speed_se3': fs.get_transformation_function("min_max_scaler"),
    'precipitaton_amount_se3': fs.get_transformation_function("min_max_scaler"),
    'total_sunshine_time_se3': fs.get_transformation_function("min_max_scaler"),
    'mean_cloud_perc_se3': fs.get_transformation_function("min_max_scaler"),
    'se4se3': fs.get_transformation_function("min_max_scaler"),
    'se3se4': fs.get_transformation_function("min_max_scaler"),
    'dk1se3': fs.get_transformation_function("min_max_scaler"),
    'se3dk1': fs.get_transformation_function("min_max_scaler"),
    'se3se2': fs.get_transformation_function("min_max_scaler"),
    'se2se3': fs.get_transformation_function("min_max_scaler"),
    'fise3': fs.get_transformation_function("min_max_scaler"),
    'se3fi': fs.get_transformation_function("min_max_scaler"),
    'no1se3': fs.get_transformation_function("min_max_scaler"),
    'se3no1': fs.get_transformation_function("min_max_scaler"),
    'fossil_gas': fs.get_transformation_function("min_max_scaler"),
    'hydro_water_reservoir': fs.get_transformation_function("min_max_scaler"),
    'nuclear': fs.get_transformation_function("min_max_scaler"),
    'other': fs.get_transformation_function("min_max_scaler"),
    'solar': fs.get_transformation_function("min_max_scaler"),
    'wind_onshore': fs.get_transformation_function("min_max_scaler")
}

In [41]:
fv = fs.create_feature_view(
    name="se3_da_pricing_model",
    query=query,
    labels=['price'],
    transformation_functions=transformations
)

Feature view created successfully, explore it at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287/fs/7235/fv/se3_da_pricing_model/version/5


In [43]:
fv.create_training_data(
    start_time = "20220101",
    end_time = "20220930",
    description='Electricity price prediction training dataset',
    data_format="csv",
    coalesce=True,
)

Training dataset job started successfully, you can follow the progress at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai/p/7287/jobs/named/se3_da_pricing_model_5_1_create_fv_td_08112022222626/executions


(1, <hsfs.core.job.Job at 0x7f3e3bdf8280>)

In [ ]:
fv.create_training_data(
    start_time = "20221001",
    end_time = "20221015",
    description='Electricity price prediction training dataset',
    data_format="csv",
    coalesce=True,
)

Training dataset job started successfully, you can follow the progress at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai/p/7287/jobs/named/se3_da_pricing_model_5_2_create_fv_td_08112022224130/executions


(2, <hsfs.core.job.Job at 0x7f3e3c70e280>)